### record rosbag
roslaunch circuit_ai turtlebot3_stage_1.launch

roslaunch turtlebot3_teleop turtlebot3_teleop_key.launch

rosbag record -a


In [2]:
import rosbag
import ros_numpy
import numpy as np
import matplotlib.pyplot as plt
import rospy
import pandas as pd
import math
import bagpy

from bagpy import bagreader
bagfile = '2021-03-16-13-17-52.bag'
b = bagreader(bagfile)
b.topic_table

[INFO]  Successfully created the data folder 2021-03-16-12-42-59.


,Topics,Types,Message Count,Frequency
0,/clock,rosgraph_msgs/Clock,68737,1039.480545
1,/cmd_vel,geometry_msgs/Twist,754,9.979690
2,/gazebo/link_states,gazebo_msgs/LinkStates,68736,174762.666667
3,/gazebo/model_states,gazebo_msgs/ModelStates,68736,1041.028543
4,/gazebo/parameter_descriptions,dynamic_reconfigure/ConfigDescription,1,NaN
5,/gazebo/parameter_updates,dynamic_reconfigure/Config,1,NaN
6,/imu,sensor_msgs/Imu,67110,1039.996033
7,/joint_states,sensor_msgs/JointState,2062,30.544014
8,/odom,nav_msgs/Odometry,2061,22.771307
9,/rosout,rosgraph_msgs/Log,12,279620.266667


In [21]:
import sensor_msgs.point_cloud2 as pc2
from sensor_msgs.msg import PointCloud2, LaserScan
import laser_geometry.laser_geometry as lg

lp = lg.LaserProjection()
bag = rosbag.Bag('2021-03-16-13-17-52.bag')

counter = 1




for topic, msg, t in bag.read_messages(topics=['/scan']):
    pc2_msg = lp.projectLaser(msg)
    xyz_array = ros_numpy.point_cloud2.pointcloud2_to_xyz_array(pc2_msg)
    xy_laser = xyz_array[:,:2]
    
    
    #print(xy_laser[:,0].shape)
    plt.figure(figsize=(11.69,8.27))
    plt.axis('equal')
    plt.plot(xy_laser[:,0], xy_laser[:,1], 'bo') 
    plt.plot(xy_laser[75,0], xy_laser[75,1], 'ro')
    plt.plot(xy_laser[105,0], xy_laser[105,1], 'go')
    alpha = math.atan2(xy_laser[75,1]-xy_laser[105,1], xy_laser[75,0]-xy_laser[105,0])
    alpha = alpha*180/math.pi
    plt.text(0.2, 0.2, alpha, fontsize=12)
    plt.arrow(0, 0, 0, 0.25, width = 0.005)
    plt.arrow(0, 0, 0.25, 0, width = 0.005)
    
    plt.legend(["laserscans"])
    plt.xlabel('Pos. x in $m$')
    plt.ylabel('Pos. y in $m$')
    filename = f"img/points{counter:04d}.png"
    plt.savefig(filename)
    # for testing: do not go through all
    #if counter > 20:
    #    break
    counter = counter+1
    #plt.show() # <-- too many (4920 messages)
    plt.close()

In [1]:
import os
# create a HTML file for easy access (without jupyter)
os.system('jupyter nbconvert --to html wall_angle.ipynb') 
#os.system('mv wall_angle.html HTML') # move to HTML folder

0

Now run /video/create_video.py to create a video from the plots

cd ../video

python3 create_video.py

In [4]:
### questions
# is this the right point for a corner?
# how about using the other points (additionally)?